In [101]:
#https://www.youtube.com/watch?v=8ZZDNd4eyVI&t=1s&ab_channel=Mr.PSolver
import numpy as np
import sympy as smp

Symbols Init

In [102]:
t, g = smp.symbols('t g')
m1, m2 = smp.symbols('m1 m2')
L1, L2 = smp.symbols('L1, L2')
T1, T2 = smp.symbols('T1, T2')

the1, the2 = smp.symbols(r'\theta_1, \theta_2', cls=smp.Function)
the1 = the1(t)
the2 = the2(t)

the1_d = smp.diff(the1, t)
the2_d = smp.diff(the2, t)
the1_dd = smp.diff(the1_d, t)
the2_dd = smp.diff(the2_d, t)

Forward Kinematics

In [103]:
x1 = L1*smp.sin(the1)
y1 = -L1*smp.cos(the1)

x2 = x1 + L2*smp.sin(the2)
y2 = y1 - L2*smp.cos(the2)

Dynamic Model

In [104]:
# Kinetic energy: it's 1/2mV^2 
K1 = 1/2 * m1 * (smp.diff(x1, t)**2 + smp.diff(y1, t)**2)
K2 = 1/2 * m2 * (smp.diff(x2, t)**2 + smp.diff(y2, t)**2)
K = K1 + K2

# potential energy
V1 = m1*g*y1
V2 = m2*g*y2
V = V1 + V2

# lagrangian 
L = K-V

Differential Equations

In [105]:
LE1 = (smp.diff(L, the1) - smp.diff(smp.diff(L, the1_d), t)).simplify() - T1
LE2 = (smp.diff(L, the2) - smp.diff(smp.diff(L, the2_d), t)).simplify() - T2

# sympy.solve() assumes all [LE1, LE2, LE3, LE4] each are equal to zero
sols = smp.solve([LE1, LE2], (the1_dd, the2_dd), simplify=True, rational=True)

thdotdot1_f = smp.lambdify((t, T1, T2, g, m1, m2, L1, L2, the1, the2, the1_d, the2_d), sols[the1_dd])
thdotdot2_f = smp.lambdify((t, T1, T2, g, m1, m2, L1, L2, the1, the2, the1_d, the2_d), sols[the2_dd])


sols = smp.solve([sols[the1_dd], sols[the2_dd]], (T1, T2), simplify=True, rational=True)

T2Eq = sols[T1]
T1Eq = sols[T2]

T1_f = smp.lambdify((t, g, m1, m2, L1, L2, the1, the2, the1_d, the2_d), T1Eq)
T2_f = smp.lambdify((t, g, m1, m2, L1, L2, the1, the2, the1_d, the2_d), T2Eq)

Constants

In [106]:
M1 = 1
M2 = 1
L1 = 1
L2 = 1
g = 9.8
Kp = 1
Kd = 1

Physics Init

In [108]:
theta1 = np.pi*3/4
theta1dot = 0
theta2 = np.pi*3/4
theta2dot = 0

previous_theta1 = theta1
previous_theta2 = theta2

previous_theta1dot = theta1dot
previous_theta2dot = theta2dot

end_effector_x = 0
end_effector_y = 0
desired_end_effector_x = 0
desired_end_effector_y = 0

t = 0
dt = 0.04

def updateTheta(t, T1_, T2_, dt, theta1, theta2, theta1dot, theta2dot):
    theta1dotdot = dz1dt_f(t, T1_, T2_, g, M1, M2, L1, L2, theta1, theta2, theta1dot, theta2dot)
    theta2dotdot = dz2dt_f(t, T1_, T2_, g, M1, M2, L1, L2, theta1, theta2, theta1dot, theta2dot)

    theta1dot = theta1dotdot*dt + theta1dot
    theta2dot = theta2dotdot*dt + theta2dot

    theta1 = theta1dot*dt + theta1
    theta2 = theta2dot*dt + theta2
    return theta1, theta2, theta1dot, theta2dot


def impedenceControl(theta1, theta2, theta1dot, theta2dot):
    
    elbow_speed = (-L1*np.sin(theta1dot), L1*np.cos(theta1dot))
    end_effector_speed = (elbow_speed[0]-L2*np.sin(theta2dot), elbow_speed[1]+L2*np.cos(theta2dot))
    
    T1_ = Kp*(end_effector_x - desired_end_effector_x) + Kd*(end_effector_speed[0] - 0)
    T2_ = Kp*(end_effector_y - desired_end_effector_y) + Kd*(end_effector_speed[1] - 0)
    
    T1_ = T1_ * L1*np.sin(theta1) + T1_ * L1*np.cos(theta1)
    T2_ = T2_ * ( -L1*np.sin(theta1) - L2*np.sin(theta2) ) + T2_ * ( L1*np.cos(theta1) + L2*np.cos(theta2) )
    
    # gravity compensation
    T1_ = T1_ + T1_f(t, g, M1, M2, L1, L2, theta1, theta2, theta1dot, theta2dot)
    T2_ = T2_ + T2_f(t, g, M1, M2, L1, L2, theta1, theta2, theta1dot, theta2dot)
    
    return T1_, T2_
    

Init OpenCV

In [109]:
import cv2
from IPython.display import clear_output

cv2.destroyAllWindows()


def forceIntroduced(event, x, y, flags, param):
    global desired_end_effector_x, desired_end_effector_y

    if event==cv2.EVENT_LBUTTONDOWN:
        desired_end_effector_x = x*2/100 - 4
        desired_end_effector_y = y*2/100 - 4
        #print("difference x", desired_end_effector_x-end_effector_x, "y", desired_end_effector_y-end_effector_y)
        clear_output(wait=True)
        
def kpChanged(arg):
    global Kp
    Kp = arg
    clear_output(wait=True)
    print("Kp", Kp, "Kd", Kd, "M1", M1, "M2", M2)

def kdChanged(arg):
    global Kd
    Kd = arg
    clear_output(wait=True)
    print("Kp", Kp, "Kd", Kd, "M1", M1, "M2", M2)

def m1Changed(arg):
    global M1
    M1 = arg
    clear_output(wait=True)
    print("Kp", Kp, "Kd", Kd, "M1", M1, "M2", M2)

def m2Changed(arg):
    global M2
    M2 = arg
    clear_output(wait=True)
    print("Kp", Kp, "Kd", Kd, "M1", M1, "M2", M2)
    
cv2.namedWindow('output', cv2.WINDOW_AUTOSIZE)
cv2.createTrackbar('Kp', 'output', 0, 10, kpChanged)
cv2.createTrackbar('Kd', 'output', 0, 10, kdChanged)
cv2.createTrackbar('M1', 'output', 0, 10, m1Changed)
cv2.createTrackbar('M2', 'output', 0, 10, m2Changed)

cv2.setTrackbarPos('Kp','output', int(Kp))
cv2.setTrackbarPos('Kd','output', int(Kd))
cv2.setTrackbarPos('M1','output', M1)
cv2.setTrackbarPos('M2','output', M2)


width = 400
height = 400

def draw(theta1, theta2):
    global end_effector_x, end_effector_y
    window = np.zeros((height, width, 3), dtype=np.uint8)
    elbow = (200-L1*np.sin(theta1)*70, 200+L1*np.cos(theta1)*70)
    end_effector = (elbow[0]-L2*np.sin(theta2)*70, elbow[1]+L2*np.cos(theta2)*70)

    elbow = (int(elbow[0]), int(elbow[1]))
    end_effector = (int(end_effector[0]), int(end_effector[1]))
    
    end_effector_x = end_effector[0]*2/100 - 4
    end_effector_y = end_effector[1]*2/100 - 4

    window = cv2.line(window, (200, 200), elbow, (255, 0, 0), 6)
    window = cv2.line(window, elbow, end_effector, (255, 0, 0), 6)
    
    window = cv2.circle(window, (int((desired_end_effector_x+4)*50), int((desired_end_effector_y+4)*50)), 5, (255, 255, 255), 2)
    return window

Kp 10 Kd 8 M1 1 M2 1


In [110]:
def reset():
    global theta1, theta1dot, theta2, theta2dot, t
    theta1 = np.pi*3/4
    theta1dot = 0
    theta2 = np.pi*3/4
    theta2dot = 0
    t = 0

while True:
    try:
        clear_output(wait=True)
        print("p", end_effector_x, end_effector_y, "dp", desired_end_effector_x, desired_end_effector_y)
        T1_, T2_ = impedenceControl(theta1, theta2, theta1dot, theta2dot)
        #T1_ = 0
        #T2_ = 0
        theta1, theta2, theta1dot, theta2dot = updateTheta(t, T1_, T2_, dt, theta1, theta2, theta1dot, theta2dot)
        window = draw(theta1, theta2)
        t = t + dt
        cv2.imshow('output', window)
        cv2.setMouseCallback('output', forceIntroduced)

    except Exception as e:
        print(e)
        reset()

    if cv2.waitKey(1) & 0xFF == ord('r'):
        print("reseting")
        reset()
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

        
cv2.destroyAllWindows()

p -0.94 -0.2200000000000002 dp 0.3200000000000003 0.0
